In [ ]:
%load_ext autoreload
%autoreload 2
%config Completer.use_jedi = False

In [ ]:
from pathlib import Path

from wikipedia_cleanup.data_filter import KeepAttributesDataFilter, generate_default_filters
from wikipedia_cleanup.predict import TrainAndPredictFramework
from wikipedia_cleanup.predictor import ZeroPredictor, OnePredictor, MeanPredictor, RandomPredictor
from wikipedia_cleanup.property_correlation import PropertyCorrelationPredictor
from wikipedia_cleanup.random_forest import RandomForestPredictor

import pandas as pd

In [ ]:
n_files = 2
n_jobs = 8
#input_path = Path("../data/test")
#input_path = Path("../../data/custom-format-default-filtered/")
#input_path = Path("/run/media/secret/manjaro-home/secret/mp-data/custom-format-default-filtered")
model = RandomForestPredictor(use_cache=False)
framework = TrainAndPredictFramework(model, group_key=['infobox_key', 'property_name'])

In [ ]:
framework.data = pd.read_csv('../../../data/popular_data_with_features2.csv')[:100000]

In [ ]:
framework.data["value_valid_from"] = pd.to_datetime(framework.data["timestamp"]).dt.tz_localize(None)
data_df = framework.data

In [ ]:
from datetime import datetime
df=framework.data
test = df[df["value_valid_from"]<datetime(2018, 9, 1)].groupby(['infobox_key', 'property_name'],sort=False).count()["value_valid_from"]
test.sort_values()

In [ ]:
# keys = test[((5<test) & (test<=10))].index.to_list()
# bucket_df = data_df[(data_df['infobox_key'] == keys[0][0]) & (data_df['property_name'] == keys[0][1])]
# for key in keys[1:]:
#     bucket_df = bucket_df.append(
#         data_df[(data_df['infobox_key'] == key[0]) & (data_df['property_name'] == key[1])])
# framework.data = bucket_df
# group_key=['infobox_key', 'property_name']
# framework.data["key"] = list(
#             zip(*[framework.data[group_key] for group_key in framework.group_key])
#         )
# framework.fit_model()
# framework.test_model(randomize=False, predict_subset=0.5)

In [ ]:
from tqdm import tqdm
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import precision_recall_fscore_support

def calc_precision(preds,y,days_diff):
    # return np.sum(np.abs(np.floor(preds)-y)<=days_diff)/len(y)
    # print(precision_recall_fscore_support(preds,y)[0])
    tp=np.sum((preds-y)==0)
    fp=np.sum((preds-y)!=0)
    return tp/(tp+fp)


def generate_predictions(key_list):
    predictions = []
    accuracys = []
    feature_importances = []
    for key in tqdm(key_list):
        sample = df[(df['infobox_key'] == key[0]) &
                        (df['property_name'] == key[1])]
        # sample = df_pop.iloc[start:start+iloc]
        # trainsize=round(len(sample)*0.8) # 20 % testdata
        sample.drop(columns=["infobox_key", "page_title",
                    "property_name", "template","Unnamed: 0","mean_change_frequency_all_previous"], inplace=True)

        trainset = sample[sample["value_valid_from"] < datetime(2018, 8, 31)]
        testset = sample[sample["value_valid_from"] >= datetime(2018, 8, 31)]
        testset = testset[testset["value_valid_from"] < datetime(2019, 9, 1)]
        if len(trainset) == 0 or len(testset) == 0:
            continue
        X_train = trainset.drop(columns=['days_until_next_change', 'timestamp','value_valid_from'])
        y_train = trainset['days_until_next_change']

        X_test = testset.drop(columns=['days_until_next_change', 'timestamp','value_valid_from'])
        y_test = testset['days_until_next_change']
        # x_last = sample.iloc[-1].drop(['days_until_next_change','timestamp']).values.reshape(1,-1)

        # x_last_timestamp = datetime.datetime.strptime(sample.iloc[-1]["timestamp"],"%Y-%m-%d").date()
        clf = RandomForestClassifier(random_state=0, n_estimators=10,max_features="auto")
        clf.fit(X_train, y_train)
        preds = clf.predict(X_test)
        predictions.append(preds)
        # predictions.append(len(y_test))
        accuracys.append(calc_precision(preds, y_test, 0))
        feature_importances.append(clf.feature_importances_)

    return predictions,y_test, accuracys, feature_importances

def generate_predictions_fw(key_list,clf):
    predictions = []
    accuracys = []
    feature_importances = []
    for key in tqdm(key_list):
        sample = df[(df['infobox_key'] == key[0]) &
                        (df['property_name'] == key[1])]
        # sample = df_pop.iloc[start:start+iloc]
        # trainsize=round(len(sample)*0.8) # 20 % testdata
        sample.drop(columns=["infobox_key", "page_title",
                    "property_name", "template","Unnamed: 0","mean_change_frequency_all_previous"], inplace=True)

        trainset = sample[sample["value_valid_from"] < datetime(2018, 8, 31)]
        testset = sample[sample["value_valid_from"] >= datetime(2018, 8, 31)]
        testset = testset[testset["value_valid_from"] < datetime(2019, 9, 1)]
        print(trainset.shape,testset.shape)
        if len(trainset) == 0 or len(testset) == 0:
            continue
        X_train = trainset.drop(columns=['days_until_next_change', 'timestamp','value_valid_from'])
        y_train = trainset['days_until_next_change']

        X_test = testset.drop(columns=['days_until_next_change', 'timestamp','value_valid_from'])
        y_test = testset['days_until_next_change']
        # x_last = sample.iloc[-1].drop(['days_until_next_change','timestamp']).values.reshape(1,-1)

        # x_last_timestamp = datetime.datetime.strptime(sample.iloc[-1]["timestamp"],"%Y-%m-%d").date()
        preds = clf.predict(X_test)
        predictions.append(preds)
        # predictions.append(len(y_test))
        accuracys.append(calc_precision(preds, y_test, 0))
        feature_importances.append(clf.feature_importances_)

    return predictions,y_test, accuracys, feature_importances



In [ ]:
# keys = [("254442624-0","num_episodes")]
keys = [("12241251-0" ,"num_episodes"),("254442624-0","num_episodes")]
cols=['day_of_year', 'day_of_month', 'day_of_week', 'month_of_year',
       'quarter_of_year', 'is_month_start', 'is_month_end', 'is_quarter_start',
       'is_quarter_end', 'days_since_last_change', 'days_since_last_2_changes',
       'days_since_last_3_changes', 'days_between_last_and_2nd_to_last_change', 'mean_change_frequency_last_3']
preds,y_test, precs,feat_imps=generate_predictions(keys)
print(precs)
# pd.Series(feat_imps[0],index=cols).sort_values(ascending=False)

In [ ]:
# # np.average(precs,weights=weights)
# preds
# y_test.to_numpy()
# ((preds[0]-y_test.to_numpy())==0).sum()#/len(y_test)
# # len(preds[0])

In [ ]:
# ttt=[3,1,1,1,1,3,1,2,4,1,3,1,1,1,1,3,1,1,1,1,3,1,1,2,1,2,2,1,1,1,1,1,4,1,1,2,1,4,1,1,1,1,1,2,2,1,1,2,2,1,3,1,1,2,1,3,1,1,2,1,3,1,2,2,1,2,2,1,1,1,1,1,1,1,1,5,1,1,1,2,1,1,4,3,1,1,1,3,1,1,1,3,1,1,4,1,1,3,3,1,1,2,1,5,1,1,1,1,3,1,1,1,3,1,1,1,3,1,1,1,1,3,1,1,1,3,1,1,2,3,3,1,1,1,3,1,3,1,1,1,2,1,1,1,1,1,1,4,2,1,1,1,3,2,1,1,1,1,1,1,2,1,1,1,3,1,1,1,1,3,1,1,1,1,3,1,1,1,3,3,1,1,3,1,1,7,1,1,1,1,3,1,1,1,1,3,1,1,1,1,3,1,1,3,1,1,3,3,1,1,1]
# ((ttt-y_test.to_numpy()[:-1])==0).sum()/len(y_test)
# framework.data["value_valid_from"] = pd.to_datetime(framework.data["value_valid_from"]).dt.tz_localize(None)

In [ ]:
# keys = test[2000<test].index.to_list()<
# bucket_df = data_df[(data_df['infobox_key'] == keys[0][0]) & (data_df['property_name'] == keys[0][1])]
bucket_df=framework.data[:10000]
for key in keys:
    bucket_df = bucket_df.append(
        data_df[(data_df['infobox_key'] == key[0]) & (data_df['property_name'] == key[1])])
framework.data = bucket_df
group_key=['infobox_key', 'property_name']
framework.data["key"] = list(
            zip(*[framework.data[group_key] for group_key in framework.group_key])
        )
framework.fit_model()
framework.test_model(randomize=False, predict_subset=1)

In [ ]:
fw_model=framework.predictor.regressors[keys[0]]
preds_fw,y_test_fw, precs_fw,feat_imps_fw=generate_predictions_fw(keys,fw_model)
print(precs_fw)

In [ ]:
print(preds_fw)
print(preds)

In [ ]:
group_key=['infobox_key', 'property_name']
framework.data["key"] = list(
            zip(*[framework.data[group_key] for group_key in framework.group_key])
        )

In [ ]:
framework.fit_model()

In [ ]:
framework.test_model(randomize=False, predict_subset=0.5)

In [ ]:
# test = df.groupby(['infobox_key', 'property_name']).count()["day_of_year"].sort_values(ascending=False)

In [ ]:
# len(test)

In [ ]:
from datetime import datetime
df=framework.data
test = df[df["value_valid_from"]<datetime(2018, 9, 1)].groupby(['infobox_key', 'property_name'],sort=False).count()["day_of_year"]

In [ ]:
keys = test[((0<test) & (test<=10))].index.to_list()
bucket_df = data_df[(data_df['infobox_key'] == keys[0][0]) & (data_df['property_name'] == keys[0][1])]
for key in keys[1:]:
    bucket_df = bucket_df.append(
        data_df[(data_df['infobox_key'] == key[0]) & (data_df['property_name'] == key[1])])
framework.data = bucket_df
framework.fit_model()
framework.test_model(randomize=False, predict_subset=0.5)


In [ ]:
framework.test_model(randomize=False, predict_subset=0.5)

In [ ]:
importances = pd.read_csv('../../../data/rf_feature_importances_1000rfs.csv',index_col=0)

In [ ]:
importances.describe()